In [1]:
import time
import json
import sys
import os
import glob
import re

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [2]:
fxc = FuncXClient()
# fxs = FuncXSerializer() # (de)serialize the funcx codes

In [3]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    from subprocess import PIPE
    
    python_path = data['python_path']
    script_path = data['script_path']
    
    #recon. script parameters
    ifpath = data['ifpath']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_ngpu = data['rec_ngpu']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    
    try:
        os.mkdir(ofpath)
    except:
        pass
    
    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' --folder='{ofpath}'"

    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr
    
func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

675a966d-45c7-4b5b-bbec-ab186e9b4cb2


In [4]:
def get_folder_paths(path):
    import glob
    import re
    
    #return f"{path}/teko007"
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join(reversed(v)).index('/'):]).group(0)))

def get_file_paths(path):
    import glob
    import re
    
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join((reversed(v))).index('/'):v.index('.')]).group(0)))

fx_func_get_file_paths_uuid = fxc.register_function(get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

4f7a2ab3-02c3-4256-91d1-0eb9b6d039dc
fa44d6b1-b59b-4052-a090-8f2c040a39e0


In [5]:
def copy_file(params):
    import os
    import shutil
    
    ifpath = params['ifpath']
    ofpath = params['ofpath']
    
    #shutil.copyfile(ifpath, ofpath)
    shutil.copytree(ifpath, ofpath)

def my_hello():
    return "Welcome to the machine"

func_cp_uuid = fxc.register_function(copy_file)
func_hl_uuid = fxc.register_function(my_hello)
print(func_cp_uuid)

d475787d-8e11-4472-a94c-1834ecdc01cf


In [6]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": True
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 1800,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 3600,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 1800,
      "End": True
    },
  }
}

In [7]:
prisma_fx_endpoint = '4bf59543-3398-42d2-9416-c628e9f5635f'

src_wf_root_path = '/prisma-data1/bicer/workflow' #'/gdata/RAVEN/bicer/2020-1/comm_33IDD/globus_automate'
src_input_folder_prefix = "input"
src_output_folder_prefix = "output"

dest_wf_root_path = '/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate'
dest_input_folder_prefix = "input"
dest_output_folder_prefix = "output"


#src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*.h5"
src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*"
rid = fxc.run(src_input_folder_paths_regex, 
              endpoint_id=prisma_fx_endpoint, 
              function_id=fx_func_get_folder_paths_uuid)
src_input_folder_paths = fxc.get_result(rid)

src_output_folder_paths = []
dest_output_folder_paths = []
dest_input_folder_paths = []
for src_input_folder_path in src_input_folder_paths:
    #print(src_input_folder_path)
    iid = re.findall(r'\d+', src_input_folder_path)
    src_output_folder_path = f"{src_wf_root_path}/{src_output_folder_prefix}/{iid[-1]}"
    src_output_folder_paths.append(src_output_folder_path)
    dest_input_folder_path = f"{dest_wf_root_path}/{dest_input_folder_prefix}/{iid[-1]}"
    dest_input_folder_paths.append(dest_input_folder_path)
    dest_output_folder_path = f"{dest_wf_root_path}/{dest_output_folder_prefix}/{iid[-1]}"
    dest_output_folder_paths.append(dest_output_folder_path)
    
    

# src_input_folder_paths: diffraction patterh files to be processed @ APS
# src_output_folder_paths: folders for reconstrcuted images after processing @ APS
# dest_input_folder_paths: diffraction patterh files to be processed @ ALCF
# dest_output_folder_paths: folders for reconstrcuted images after processing @ ALCF

for (src_input_folder_path, src_output_folder_path, dest_input_folder_path, dest_output_folder_path ) in zip(src_input_folder_paths, src_output_folder_paths, dest_input_folder_paths, dest_output_folder_paths):
    print(f"Source input folder: {src_input_folder_path}; Source output folder: {src_output_folder_path}")
    print(f"Dest. input folder: {dest_input_folder_path}; Dest. output folder: {dest_output_folder_path}")
    print()

Source input folder: /prisma-data1/bicer/workflow/input/scan192; Source output folder: /prisma-data1/bicer/workflow/output/192
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/192; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/192

Source input folder: /prisma-data1/bicer/workflow/input/scan193; Source output folder: /prisma-data1/bicer/workflow/output/193
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/193; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/193

Source input folder: /prisma-data1/bicer/workflow/input/scan194; Source output folder: /prisma-data1/bicer/workflow/output/194
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/194; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/194

Source input folder: /prisma-data1/bicer/workflow/input/scan195; Source output folder

In [8]:
theta_fx_endpoint = '7f42390d-849a-42a7-905c-db6b22af28f7' #'f765db7a-038c-47ea-9176-d81de31c054f' #FuncX endpoint

#src_endpoint = '9c9cb97e-de86-11e6-9d15-22000a1e3b52' #'aps#data' # Voyager
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' #'hostel' #aps/workstation
dest_endpoint = 'e09e65f5-6d04-11e5-ba46-22000b92c6ec' #'08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta' # Theta DTN

# Ptycho recon params
script_path = '/home/bicer/projects/tike/scripts/mpi_tike-recon.py'
python_path = "/home/bicer/miniconda3/envs/ptycho/bin/python"

rec_alg = 'cgrad'
rec_nmodes = 8
rec_upd_pos = False
rec_niter = 10
rec_output_freq = 5
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 1
rec_use_mpi = False

flow_inputs = []

for (src_input_folder_path, src_output_folder_path, 
     dest_input_folder_path, dest_output_folder_path ) in zip(
    src_input_folder_paths, src_output_folder_paths, 
    dest_input_folder_paths, dest_output_folder_paths):
    
    flow_input = {
        "input": {
            "source_endpoint": f"{src_endpoint}",
            "source_path": f"{src_input_folder_path}",
            "dest_endpoint": dest_endpoint,
            "dest_path": f"{dest_input_folder_path}",

            "result_path": f"{dest_output_folder_path}",#f"{dest_resultpath}/out_{src_filename}",
            "source_result_path": f"{src_output_folder_path}", #/out_{src_filename}",
            "fx_ep": f"{theta_fx_endpoint}",
            "fx_id": f"{func_cp_uuid}", #f"{func_hl_uuid}" #
            #"fx_id": f"{func_ptycho_uuid}",
            #"params": {'ifpath': f"{dest_input_folder_path}",
            #           'ofpath': f"{dest_output_folder_path}/",
            #           'script_path': script_path,
            #           'python_path': python_path,
            #           'rec_alg': rec_alg,
            #           'rec_nmodes': rec_nmodes,
            #           'rec_upd_pos': rec_upd_pos,
            #           'rec_niter': rec_niter,
            #           'rec_output_freq': rec_output_freq,
            #           'rec_recover_psi': rec_recover_psi,
            #           'rec_recover_probe': rec_recover_probe,
            #           'rec_recover_positions': rec_recover_positions,
            #           'rec_model': rec_model,
            #           'rec_ngpu': rec_ngpu,
            #           'rec_use_mpi': rec_use_mpi}
            "params": {"ifpath": f"{dest_input_folder_path}",
                       "ofpath": f"{dest_output_folder_path}"}
        }
    }
    flow_inputs.append(flow_input)

#print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")
#print(f"recon file:{dest_filepath}/{src_filename} output:{dest_resultpath}/")
#print(f"transfer file from {dest_endpoint}:{dest_resultpath} to {src_endpoint}#{src_result_path}/")

In [9]:
print(len(flow_inputs))

168


In [10]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple ptycho data analysis flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

#print(flow)
#print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

In [11]:
ga_flow_actions = {}
for flow_input in flow_inputs:
    flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
    ga_flow_actions[flow_action['action_id']] = flow_action
    print(f'Flow is initiated: {flow_action}')

while any(ga_flow_action['status'] == 'ACTIVE' for ga_flow_action in ga_flow_actions.values()):
    time.sleep(10)
    for ga_flow_action_id, ga_flow_action in ga_flow_actions.items():
        ga_flow_actions[ga_flow_action_id] = flows_client.flow_action_status(flow_id, flow_scope, ga_flow_action_id)
        print(f"Flow status: {ga_flow_action_id}: {ga_flow_action['status']}")
    print()

for ga_flow_action_id, ga_flow_action in ga_flow_actions.items():
    print(f"Flow status: {ga_flow_action_id}: {ga_flow_action['status']}")

Starting login with Globus Auth, press ^C to cancel.
Flow is initiated: GlobusHTTPResponse({'action_id': '0fdd462f-1e9e-41af-adfe-a20ef8e5aa27', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'details': {'code': 'ActionStarted', 'description': 'State Transfer of type Action started', 'details': {'input': {'destination_endpoint_id': 'e09e65f5-6d04-11e5-ba46-22000b92c6ec', 'source_endpoint_id': 'dd916908-0072-11e7-badc-22000b9a448b', 'transfer_items': [{'destination_path': '/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/192', 'recursive': True, 'source_path': '/prisma-data1/bicer/workflow/input/scan192'}]}, 'state_name': 'Transfer', 'state_type': 'Action'}, 'time': '2021-03-10T22:26:16.808000+00:00'}, 'start_time': '2021-03-10T22:26:16.773000+00:00', 'status': 'ACTIVE'})
Flow is initiated: GlobusHTTPResponse({'action_id': '8d7cf971-093b-4de6-9a25-9c6eb013248d', 'completion_time': 'None', 'created_by': 'urn:globus:

Flow is initiated: GlobusHTTPResponse({'action_id': 'dc72e20c-e4eb-44fb-9ac3-229587b697b0', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'details': {'code': 'ActionStarted', 'description': 'State Transfer of type Action started', 'details': {'input': {'destination_endpoint_id': 'e09e65f5-6d04-11e5-ba46-22000b92c6ec', 'source_endpoint_id': 'dd916908-0072-11e7-badc-22000b9a448b', 'transfer_items': [{'destination_path': '/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/204', 'recursive': True, 'source_path': '/prisma-data1/bicer/workflow/input/scan204'}]}, 'state_name': 'Transfer', 'state_type': 'Action'}, 'time': '2021-03-10T22:26:33.203000+00:00'}, 'start_time': '2021-03-10T22:26:33.158000+00:00', 'status': 'ACTIVE'})
Flow is initiated: GlobusHTTPResponse({'action_id': '9cc1b935-2dd6-416c-bfcd-7e861a25b86b', 'completion_time': 'None', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 

KeyboardInterrupt: 

In [ ]:
#flows_client.flow_action_cancel(flow_id, flow_scope, '5e948f48-c3c3-4328-8d37-b5d1bc82df4c')

In [12]:
for ga_flow_action_id in ga_flow_actions.keys():
    flows_client.flow_action_cancel(flow_id, flow_scope, ga_flow_action_id)

In [ ]:
#st = flows_client.flow_action_status(flow_id, flow_scope, '08925f04-569f-11e7-bef8-22000b9a448b')
#print(st['status'])